In [ ]:
import pandas as pd
import xgboost as xgb
import bz2
import _pickle as pickle
%matplotlib inline
from params import *

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 150)

In [18]:
train = pd.concat([pd.read_pickle('data/aggregation/train.pkl', compression='gzip'),
                pd.read_pickle('data/cross/train.pkl', compression='gzip'),
                pd.read_pickle('data/double_cross/train.pkl', compression='gzip'),
                pd.concat([pd.read_pickle('data/folds/X_test_fold_0.csv', compression='gzip'),
                               pd.read_pickle('data/folds/X_test_fold_1.csv', compression='gzip'),
                               pd.read_pickle('data/folds/X_test_fold_2.csv', compression='gzip'),
                               pd.read_pickle('data/folds/X_test_fold_3.csv', compression='gzip'),
                               pd.read_pickle('data/folds/X_test_fold_4.csv', compression='gzip')],
                            axis=0)],
                axis = 1)

In [4]:
print (train.shape)
train = train[list(set(train.columns.values) - set(col_nuls))]
print (train.shape)

(135155, 486)
(135155, 258)


In [ ]:
with bz2.BZ2File('models/model_xgb.pbz2', 'r') as f:
    model_fit = pickle.load(f)

In [1]:
d1 = model_fit.get_score(importance_type='gain')
d2 = model_fit.get_score(importance_type='weight')
features = pd.concat([pd.Series(d1,name='gain').to_frame(),pd.Series(d2,name='weight').to_frame()],axis=1) \
 .sort_values('weight', ascending=False) 

NameError: name 'model_fit' is not defined

In [8]:
features

,gain,weight
sum_ones_0:6,28.708842,176
MEAN_sum_ones_0:6_DEF,28.377723,35
sum_ones_0:3,28.075857,86
mean_CREDIT_CURRENCY_lastcredit_time_rur,26.930367,252
pca_2,26.200334,264
sum_ones_0:12,24.601241,124
mean_lastcredit_time,19.193923,110
pca_1,19.019722,158
mean_CREDIT_ACTIVE_lastcredit_time_2,18.839100,1
min_CREDIT_ACTIVE_lastcredit_time_1,18.589405,224


In [36]:
import numpy as np
SPLITS=5
for fold in np.arange(SPLITS):

    X_train = pd.read_pickle('data/folds/X_train_fold_' + str(fold) + '.csv', compression='gzip')
    X_test = pd.read_pickle('data/folds/X_test_fold_' + str(fold) + '.csv', compression='gzip')
    y_train = pd.read_pickle('data/folds/y_train_fold_' + str(fold) + '.csv', compression='gzip')
    y_test = pd.read_pickle('data/folds/y_test_fold_' + str(fold) + '.csv', compression='gzip')

    X_train = pd.merge(train, X_train, how='inner', left_index=True, right_index=True)
    X_test = pd.merge(train, X_test, how='inner', left_index=True, right_index=True)

    #xgb_test_global = xgb.DMatrix(test.reindex(columns=X_train.columns))
    xgb_train = xgb.DMatrix(X_train, label=y_train)
    xgb_test = xgb.DMatrix(X_test, label=y_test)

    watchlist = [(xgb_train, 'train'), (xgb_test, 'eval')]
    
    eval_result = {}

    model_fit = xgb.train(params_xgb, xgb_train, num_boost_round=1388,
                          evals=watchlist,
                          maximize=False,
                          verbose_eval=False,
                          #early_stopping_rounds = 200,
                          callbacks=[xgb.callback.record_evaluation(eval_result)]
                          )

    #scores_train.loc[X_test.index, 'xgb_predict'] = model_fit.predict(xgb_test)
    #scores_test['xgb_predict_'+str(fold)] = model_fit.predict(xgb_test_global)

    #scores[fold] = eval_result['eval']['auc']
    
    with bz2.BZ2File('models/model'+str(fold)+'_xgb.pbz2', 'w') as f:
        pickle.dump(model_fit, f)

In [17]:
t_feature = features.iloc[len(features)-1].name

In [34]:
features = train.columns.values[:]

In [35]:
features

array(['sum_zeros_0:3', 'sum_zeros_0:6', 'sum_zeros_0:12',
       'sum_zeros_0:24', 'sum_zeros_0:36', 'sum_zeros_0:48',
       'sum_zeros_0:60', 'sum_ones_0:3', 'sum_ones_0:6', 'sum_ones_0:12',
       'sum_ones_0:24', 'sum_ones_0:36', 'sum_ones_0:48', 'sum_ones_0:60',
       'sum_twos_0:3', 'sum_twos_0:6', 'sum_twos_0:12', 'sum_twos_0:24',
       'sum_twos_0:36', 'sum_twos_0:48', 'sum_twos_0:60', 'sum_threes_0:3',
       'sum_threes_0:6', 'sum_threes_0:12', 'sum_threes_0:24',
       'sum_threes_0:36', 'sum_threes_0:48', 'sum_threes_0:60',
       'sum_fours_0:3', 'sum_fours_0:6', 'sum_fours_0:12',
       'sum_fours_0:24', 'sum_fours_0:36', 'sum_fours_0:48',
       'sum_fours_0:60', 'sum_fives_0:3', 'sum_fives_0:6',
       'sum_fives_0:12', 'sum_fives_0:24', 'sum_fives_0:36',
       'sum_fives_0:48', 'sum_fives_0:60', 'sum_bads_0:3', 'sum_bads_0:6',
       'sum_bads_0:12', 'sum_bads_0:24', 'sum_bads_0:36', 'sum_bads_0:48',
       'sum_bads_0:60', 'sum_base_0:3', 'sum_base_0:6', 'sum_base

In [28]:
import numpy as np
SPLITS=5
train = pd.concat([pd.read_pickle('data/aggregation/train.pkl', compression='gzip'),
                pd.read_pickle('data/cross/train.pkl', compression='gzip'),
                pd.read_pickle('data/double_cross/train.pkl', compression='gzip'),
                pd.concat([pd.read_pickle('data/folds/X_test_fold_0.csv', compression='gzip'),
                               pd.read_pickle('data/folds/X_test_fold_1.csv', compression='gzip'),
                               pd.read_pickle('data/folds/X_test_fold_2.csv', compression='gzip'),
                               pd.read_pickle('data/folds/X_test_fold_3.csv', compression='gzip'),
                               pd.read_pickle('data/folds/X_test_fold_4.csv', compression='gzip')],
                            axis=0)],
                axis = 1)
for fold in np.arange(SPLITS):
    
    X_train = pd.read_pickle('data/folds/X_train_fold_' + str(fold) + '.csv', compression='gzip')
    X_test = pd.read_pickle('data/folds/X_test_fold_' + str(fold) + '.csv', compression='gzip')
    y_train = pd.read_pickle('data/folds/y_train_fold_' + str(fold) + '.csv', compression='gzip')
    y_test = pd.read_pickle('data/folds/y_test_fold_' + str(fold) + '.csv', compression='gzip')

    X_train = pd.merge(train, X_train, how='inner', left_index=True, right_index=True)
    X_test = pd.merge(train, X_test, how='inner', left_index=True, right_index=True)
    
    np.random.shuffle(X_test[t_feature].values)
    
    #xgb_test_global = xgb.DMatrix(test.reindex(columns=X_train.columns))
    xgb_train = xgb.DMatrix(X_train, label=y_train)
    xgb_test = xgb.DMatrix(X_test, label=y_test)

    watchlist = [(xgb_train, 'train'), (xgb_test, 'eval')]
    
    eval_result = {}

    with bz2.BZ2File('models/model'+str(fold)+'_xgb.pbz2', 'r') as f:
        model_fit = pickle.load(f)
    
        
        
    

In [30]:
np.random.shuffle(X_test[t_feature].values)